In [2]:
import os
import pickle

import numpy as np
import pandas as pd

# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential
# from keras.models import model_from_yaml
# from keras.layers import Conv2D, MaxPooling2D
# from keras.layers import Activation, Dropout, Flatten, Dense
# from keras import backend as K

# from keras.applications import VGG16
# from keras.applications.vgg16 import preprocess_input
# from keras.preprocessing import image as kimage
# from keras.callbacks import EarlyStopping
# import sklearn.metrics as skmetrics
from sklearn.metrics import matthews_corrcoef
import matplotlib.pyplot as plt
%matplotlib inline

# Load train/validation probabilities and actual values

In [3]:
train_probs = np.load('data/train_probs.npy')
val_probs = np.load('data/val_probs.npy')
train_multilabels = np.load('data/train_multilabels.npy')
val_multilabels = np.load('data/val_multilabels.npy')

## Find the optimum threshold values on the training set

In [4]:
out = train_probs
y_test = train_multilabels

In [5]:
acc = []
accuracies = []
best_threshold = np.zeros(out.shape[1])
threshold = np.arange(0.1, 0.9, step=0.05)
for i in range(out.shape[1]):
    y_prob = np.array(out[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append(matthews_corrcoef(y_test[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []

/Users/harryxue/anaconda/envs/keras/lib/python2.7/site-packages/sklearn/metrics/classification.py:516: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(var_yt * var_yp)


In [6]:
gitbest_threshold

array([ 0.2 ,  0.2 ,  0.25,  0.2 ,  0.25,  0.25,  0.3 ,  0.2 ,  0.4 ,
        0.1 ,  0.3 ,  0.15,  0.25,  0.25,  0.15,  0.3 ,  0.25,  0.25,  0.25])

In [7]:
val_pred = np.array([[int(val_probs[i,j]>=best_threshold[j]) for j in range(val_probs.shape[1])] for i in range(len(val_probs))])

In [20]:
val_pred.shape

(1160, 19)

In [11]:
# Save thresholded predictions
np.save(open('data/val_pred_thresholded.npy', 'wb'), val_pred)

In [26]:
from sklearn import metrics as skmetrics

In [27]:
print 'Neural Net Predictions'
print 'hamming loss:', skmetrics.hamming_loss(val_pred, val_multilabels)
print 'jaccard similarity:', skmetrics.jaccard_similarity_score(val_pred, val_multilabels)
print 'zero one loss:', skmetrics.zero_one_loss(val_pred, val_multilabels)
print 'exact match ratio:', skmetrics.accuracy_score(val_pred, val_multilabels)

Neural Net Predictions
hamming loss: 0.197277676951
jaccard similarity: 0.242022685169
zero one loss: 0.978448275862
exact match ratio: 0.0215517241379


In [28]:
all_zeros = np.zeros((1160, 19))

In [29]:
print 'hamming loss:', skmetrics.hamming_loss(all_zeros, val_multilabels)


hamming loss: 0.138520871143


$$x - xz + \log(1 + \exp(-x))$$